**IMPORTS :** 

In [ ]:
import os
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

**Récupération du corpus** (pour qu'on puisse l'utiliser avec sklearn)

In [ ]:
dossier_train = "../../deft09/corpus_train"
dossier_test = "../../deft09/corpus_test"

In [ ]:
def recup_train_data(dossier) : 
    """
    extraction du texte et des étiquettes des documents XML d'un répertoire.
    """
    textes = []
    labels = []
    for fichier in os.listdir(dossier) : 
        if fichier.endswith(".xml"):
            chemin_fichier = os.path.join(dossier, fichier)
            tree = ET.parse(chemin_fichier) #analyse et parsing du fichier
            root = tree.getroot() #obtient l'élément racine de l'arbre XML (le noeud principal de l'arbre)

            for doc in root.findall("doc") : 
                intervention = doc.find(".//texte/p")
                if intervention is not None and intervention.text is not None : 
                 texte = intervention.text
                else : 
                    texte = ""
                parti = doc.find(".//EVALUATION/EVAL_PARTI/PARTI")
                if parti is not None and "valeur" in parti.attrib : 
                    label = parti.get("valeur")
                else : 
                    label = "Unknown"

                textes.append(texte)
                labels.append(label)
                
    return textes, labels


def recup_test_data(dossier) : 
    """
    extraction du texte des documents XML d'un répertoire.
    """
    textes = []
    for fichier in os.listdir(dossier) : 
        if fichier.endswith(".xml"):
            chemin_fichier = os.path.join(dossier, fichier)
            tree = ET.parse(chemin_fichier) #analyse et parsing du fichier
            root = tree.getroot() #obtient l'élément racine de l'arbre XML (le noeud principal de l'arbre)

            for doc in root.findall("doc") : 
                intervention = doc.find(".//texte/p")
                if intervention is not None and intervention.text is not None : 
                 texte = intervention.text
                else : 
                    texte = ""

                textes.append(texte)
    
    return textes

In [ ]:
textes_train, labels_train = recup_train_data(dossier_train)
textes_test = recup_test_data(dossier_test)

**Vectorisation TFIDF** (+enlever les stopwords)

In [ ]:
stopwords_list = stopwords.words("english") + stopwords.words("french") + stopwords.words("italian")

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords_list)
X_train = tfidf.fit_transform(textes_train)
X_test = tfidf.transform(textes_test)

## KNeighbors Classifier

In [1]:
param_gridKNN = {
    "n_neighbors" : [1, 2, 5, 10],
    "weights" : ["uniform", "distance"], 
}

#Initialisation
model_KNN = KNeighborsClassifier()
KNN_grid = GridSearchCV(model_KNN, param_gridKNN, cv=5, verbose=1)

#Entraînement du modèle
KNN_grid.fit(X_train, labels_train)

#Afficher les meilleurs paramètres
print("Meilleurs paramètres : ", KNN_grid.best_params_)

#Prédiction avec les meilleurs paramètres obtenus
best_KNN = KNN_grid.best_estimator_
y_pred_best_KNN = best_KNN.predict(X_test)

#Résultats
print("Résultats KNeighbors Classifier : ")
print("Rapport de Classification KNN:\n")
print(classification_report(y_test, y_pred_best_KNN))

NameError: name 'KNeighborsClassifier' is not defined

## Multinomial Naive Bayes

In [ ]:
param_gridMultiNB = {
    "alpha": [0.05, 0.2, 0.7, 1.0, 2],
    "fit_prior": [True, False]
}

#Initialisation
model_MultiNB = MultinomialNB()
multiNB_grid = GridSearchCV(model_MultiNB, param_gridMultiNB, cv=5, verbose=1 )

#Entraînement du modèle
multiNB_grid.fit(X_train, labels_train)

#Afficher les meilleurs paramètres
print("Meilleurs paramètres : ", multiNB_grid.best_params_)

#Prédiction avec les meilleurs paramètres obtenus
best_MultiNB = multiNB_grid.best_estimator_
y_pred_best_MultiNB = best_MultiNB.predict(X_test)

#Résultats
print("Résultat Multinomial Naive Bayes : ")
print("Rapport de Classification MultiNB:\n")
print(classification_report(y_test, y_pred_best_MultiNB))

## Complement Naive Bayes

In [ ]:
param_gridComplNB = {
    "alpha": [0.05, 0.2, 0.7, 1.0, 2],
    "fit_prior": [True, False],
    "norm": [True, False]
}

#Initialisation
model_ComplNB = ComplementNB()
ComplNB_grid = GridSearchCV(model_ComplNB, param_gridComplNB, cv=5, verbose=1 )

#Entraînement du modèle
ComplNB_grid.fit(X_train, labels_train)

#Afficher les meilleurs paramètres
print("Meilleurs paramètres : ", ComplNB_grid.best_params_)

#Prédiction avec les meilleurs paramètres obtenus
best_ComplNB = ComplNB_grid.best_estimator_
y_pred_best_ComplNB = best_ComplNB.predict(X_test)

#Résultats
print("Résultat Complement Naive Bayes : ")
print("Rapport de Classification ComplNB:\n")
print(classification_report(y_test, y_pred_best_ComplNB))

## COMPARAISON DES PERFORMANCES DES MODÈLES

In [ ]:
resultats = pd.DataFrame({
    "Modèle": ["KNeighbors", "Multinomial Naive Bayes", "Complement Naive Bayes"],
    "Accuracy" : 
        [accuracy_score(y_test, y_pred_best_KNN),
        accuracy_score(y_test, y_pred_best_MultiNB),
        accuracy_score(y_test, y_pred_best_ComplNB)],
    "Précision": 
        [precision_score(y_test, y_pred_best_KNN, average='weighted'),
        precision_score(y_test, y_pred_best_MultiNB, average='weighted'),
        precision_score(y_test, y_pred_best_ComplNB, average='weighted')],
    "Rappel" : 
        [recall_score(y_test, y_pred_best_KNN, average='weighted'),
        recall_score(y_test, y_pred_best_MultiNB, average='weighted'),
        recall_score(y_test, y_pred_best_ComplNB, average='weighted')],
    "F-mesure" : 
        [f1_score(y_test, y_pred_best_KNN, average='weighted'),
        f1_score(y_test, y_pred_best_MultiNB, average='weighted'),
        f1_score(y_test, y_pred_best_ComplNB, average='weighted')]
})

print(resultats)